In [1]:
import numpy as np
import math as m
import scipy as sc
import pandas as pd
from sklearn.datasets import make_blobs
import plotly.express as px

import logging
import sys

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

COLORS = [
    "#f7dc6f",
    "#82e0aa",
    "#f1948a",
    "#499cef",
    "#f5b041",
    "#a569bd",
    "#e74c3c",
    "#2ecc71",
    "#3498db",
    "#e67e22",
    "#9b59b6",
    "#1abc9c",
    "#34495e",
    "#d35400",
    "#c0392b",
    "#16a085",
    "#2980b9",
    "#8e44ad",
]

def twospirals(n_points, noise=.5):
    n = np.sqrt(np.random.rand(n_points,1)) * 780 * (2*np.pi)/360
    d1x = -np.cos(n)*n + np.random.rand(n_points,1) * noise
    d1y = np.sin(n)*n + np.random.rand(n_points,1) * noise
    return (np.vstack((np.hstack((d1x,d1y)),np.hstack((-d1x,-d1y)))), np.hstack((np.zeros(n_points, dtype=int),np.ones(n_points, dtype=int))))

def plot_dataframe(df, title='', x='x', y='y', label='label'):
    df[label] = df[label].astype(str)
    fig = px.scatter(
        df, 
        x=x, 
        y=y, 
        symbol=label,
        color=label,
        color_discrete_sequence= COLORS, 
        title=title
    )
    fig.update_traces(marker=dict(size=5, symbol='circle'))

    return fig

def plot_centroid(fig, centroid, colors = ['#a569bd'], marker_size=10, symbol = 'star', name = 'Centroid'):
    fig.add_scatter(
        x=[centroid[0]], 
        y=[centroid[1]], 
        mode='markers',
        marker=dict(
            size = marker_size,
            color = colors,
            symbol = symbol,
        ), 
        name=name
    )
    return fig

def create_constraints(labels, probability=0.01, seed=0):
    n_points = len(labels)
    constraints = np.zeros((n_points, n_points), dtype=int)
    state = np.random.RandomState(seed=seed)
    for i in range(n_points):
        for j in range(i +1, n_points):
            if state.rand() < probability:
                if labels[i] == labels[j]:
                    constraints[i, j] = 1
                    constraints[j, i] = 1
                elif labels[i] != labels[j]:
                    constraints[i, j] = -1
                    constraints[j, i] = -1
    return constraints

In [2]:
X, y = make_blobs(n_samples=300, centers=10, cluster_std=1.0, random_state=42)
constraints = create_constraints(y, probability=0.01, seed=42)

In [3]:
from clustlib.nonparam.tvclust import TVClust
tvclust = TVClust(
    n_clusters=10, 
    constraints=constraints, 
    max_iter=100, 
    tol=1e-4,
)
tvclust.fit(X)

2025-06-02 18:21:44,628 - DEBUG - Initializing parameters for TVClust, n_clusters=10, p=2
2025-06-02 18:21:44,629 - DEBUG - Covariance inverse: (10, 2, 2)
2025-06-02 18:21:44,629 - DEBUG - Iteration 1/100
2025-06-02 18:21:44,629 - DEBUG - Updating responsibilities
2025-06-02 18:21:44,629 - DEBUG - Calculating the determinant of the covariance
2025-06-02 18:21:44,633 - DEBUG - Updating gamma
2025-06-02 18:21:44,633 - DEBUG - Updating beta
2025-06-02 18:21:44,633 - DEBUG - Updating mu
2025-06-02 18:21:44,634 - DEBUG - Updating W
2025-06-02 18:21:44,635 - DEBUG - Updating nu
2025-06-02 18:21:44,635 - DEBUG - Updating prior
2025-06-02 18:21:44,638 - DEBUG - Calculating the determinant of the covariance
2025-06-02 18:21:44,639 - DEBUG - Iteration 2/100
2025-06-02 18:21:44,639 - DEBUG - Updating responsibilities
2025-06-02 18:21:44,640 - DEBUG - Calculating the determinant of the covariance
2025-06-02 18:21:44,643 - DEBUG - Updating gamma
2025-06-02 18:21:44,644 - DEBUG - Updating beta
2025-

/Users/ismaelmarin/Documents/Projects/clustlib/clustlib/nonparam/tvclust.py:192: RuntimeWarning: invalid value encountered in subtract
  alpha = np.concatenate(((phi_alpha - phi_alpha_beta)[:-1], [0]))
/Users/ismaelmarin/Documents/Projects/clustlib/clustlib/nonparam/tvclust.py:193: RuntimeWarning: invalid value encountered in subtract
  beta = np.concatenate(([0], np.cumsum(phi_beta - phi_alpha_beta)[:-1]))


LinAlgError: Singular matrix

In [ ]:
p = 2
K = 10
# arr = np.tile(np.arange(1, p + 1) * 0.5, (10, 1))

# nu = np.repeat(p, 10)
# nu = np.random.rand(10) * p

# (nu[:, np.newaxis] + 1) - arr
# np.tile(np.identity(p), (10, 1, 1))
muQ = np.random.randn(p, 1, K)
muQ = np.reshape(muQ, (K, p))
cov = np.tile(np.identity(p), (10, 1, 1))
print(muQ[0, :])
diff = X[5] - muQ[0, :]
print(diff)
print (np.linalg.multi_dot([diff, cov[0, :, :], diff.T]))
diff = (X - muQ[0, :])
np.diag(diff.dot(cov[0, :, :]).dot(diff.T))[5]


[0.5315034  1.09611952]
[3.76566092 0.07477289]
14.185793180911649


14.185793180911649

In [ ]:
p = 2
diff = X - tvclust.centroids[0]
print(diff)
cov = np.tile(np.identity(p), (10, 1, 1))
print(cov[0].shape)
np.linalg.multi_dot([diff, cov[0], diff.T]).shape


[[-1.77073104e+00  9.18565441e+00]
 [ 7.52117345e+00 -5.56987578e+00]
 [-8.06688310e+00 -6.38745750e+00]
 [-4.15410319e+00 -4.12435029e-01]
 [-9.31327307e+00  6.67019368e+00]
 [ 4.29716432e+00  1.17089241e+00]
 [ 5.88972015e+00 -5.60282400e+00]
 [ 2.46611966e+00  4.93608561e+00]
 [-6.62913434e+00 -6.53366138e+00]
 [-3.07946303e+00 -6.34578222e-01]
 [-7.12501531e+00 -7.63384576e+00]
 [ 1.81417798e+00  3.66845062e+00]
 [-3.27656268e+00 -1.16639143e+00]
 [-4.67635902e+00 -5.45027005e+00]
 [-2.14780202e+00  1.05523227e+01]
 [-9.34334354e+00  8.89125387e+00]
 [ 3.64934251e+00  1.40687195e+00]
 [ 6.37944598e+00 -5.03567553e+00]
 [-2.96983639e+00  1.00714084e+01]
 [-7.75527651e+00  8.37732497e+00]
 [-3.98771961e+00  8.29444192e+00]
 [-6.47582871e+00 -6.55287940e+00]
 [-2.98837186e+00  8.82862715e+00]
 [-9.36421763e+00  9.41078944e+00]
 [-4.91493707e-01 -2.81977934e+00]
 [ 5.58123239e+00 -5.89559727e+00]
 [ 7.82829294e+00 -6.22239344e+00]
 [ 3.72045460e+00  3.52310409e+00]
 [ 4.44751787e+00  2

(300, 300)

In [ ]:
matrix = np.random.rand(3000).reshape((10, 300))
array = np.ones(10) * 2
print(matrix)
print("----------------------------")
print(array)
print("----------------------------")
print((array[np.newaxis, :] * matrix.T).T)

[[0.52846722 0.43539218 0.99085482 ... 0.43854    0.45684973 0.33635112]
 [0.652463   0.53634083 0.61143754 ... 0.45550101 0.49025434 0.82764351]
 [0.59169299 0.94973051 0.28670605 ... 0.71162921 0.40057635 0.18560491]
 ...
 [0.79899238 0.03596366 0.86822177 ... 0.38461921 0.43203177 0.70147633]
 [0.15857223 0.47974517 0.29314731 ... 0.99768838 0.62222329 0.31762993]
 [0.71372878 0.02936146 0.5599555  ... 0.05086725 0.54777202 0.30010479]]
----------------------------
[2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
----------------------------
[[1.05693443 0.87078437 1.98170963 ... 0.87707999 0.91369946 0.67270224]
 [1.304926   1.07268167 1.22287507 ... 0.91100202 0.98050868 1.65528702]
 [1.18338599 1.89946103 0.57341211 ... 1.42325841 0.80115271 0.37120982]
 ...
 [1.59798476 0.07192733 1.73644353 ... 0.76923843 0.86406355 1.40295265]
 [0.31714445 0.95949033 0.58629461 ... 1.99537675 1.24444659 0.63525986]
 [1.42745756 0.05872292 1.11991099 ... 0.10173451 1.09554403 0.60020958]]


In [ ]:
concentration = tvclust.concentration()

2025-06-02 16:40:52,432 - DEBUG - Calculating the determinant of the covariance


In [ ]:
expected = tvclust.expected_distance()

In [ ]:
from scipy.special import digamma as phi

trust = (concentration[:, np.newaxis] - expected) * 0.5
gamma = np.random.rand(10 * 2).reshape((10, 2))
phi_alpha_beta = phi(np.sum(gamma, axis = 1))
phi_alpha = phi(gamma[:, 0])
phi_beta = phi(gamma[:, 1])

NameError: name 'concentration' is not defined

In [ ]:
alpha = np.concatenate(((phi_alpha - phi_alpha_beta)[:-1], [0]))

In [ ]:
beta = np.concatenate(([0], np.cumsum(phi_beta - phi_alpha_beta)[:-1]))
alpha + beta

array([ -4.32119655,  -1.36124161, -29.87323537, -36.62062387,
       -37.59841506, -31.67394159, -33.5198809 , -34.6610947 ,
       -41.33150405, -41.80004284])

In [ ]:
ri = np.zeros(10)
for i in range(10):
    if i < 9:
        tmp = phi(gamma[i, 0]) - phi(np.sum(gamma[i, :]))
        print(f"i: {i}, tmp: {tmp}")
    elif i == 9:
        tmp = 0
    
    if i > 0:
        for j in range(i):
            tmp += phi(gamma[j, 1]) - phi(np.sum(gamma[j, :]))
    
    ri[i] = tmp

print(ri)

i: 0, tmp: -3.1416518255235655
i: 1, tmp: -3.086101785851395
i: 2, tmp: -3.4471469549005587
i: 3, tmp: -1.6915165653679092
i: 4, tmp: -4.396456323174174
i: 5, tmp: -3.1824978481009922
i: 6, tmp: -2.542464788266251
i: 7, tmp: -4.672661478904506
i: 8, tmp: -1.064150342249795
[ -3.14165183  -4.07815223  -5.14566741 -14.62382078 -18.17597105
 -18.1879212  -18.13916927 -20.80605492 -18.19703128 -18.22124312]


In [ ]:
trust = (tvclust.concentration()[:, np.newaxis] - tvclust.expected_distance()) * 0.5

2025-06-02 16:41:06,718 - DEBUG - Calculating the determinant of the covariance


In [ ]:
trust.T[0] + alpha + beta

array([ -81.65055026, -107.75874307, -104.31278826, -105.83313591,
       -124.51028348, -117.22439396, -141.39536885, -118.13597224,
       -150.78481969, -130.81393666])

In [ ]:
ejemplo = tvclust.constraints[0]

ml_corr = tvclust.ml_correction()
cl_corr = tvclust.cl_correction()

2025-06-02 17:07:32,084 - DEBUG - 
            ML correction: 
                phi_alpha=-0.5772156649015329, 
                phi_beta=-0.5772156649015329, 
                phi_alpha_beta_p=2.3517525890667215, 
                phi_alpha_beta_q=2.3517525890667215
            
2025-06-02 17:07:32,087 - DEBUG - 
            CL correction: 
                phi_alpha=2.251752589066721, 
                phi_beta=2.251752589066721, 
                phi_alpha_beta_p=2.3517525890667215, 
                phi_alpha_beta_q=2.3517525890667215
            


In [ ]:
ejemplo[np.where(ejemplo <= 0)] = 0.
corrections = ejemplo * ml_corr + (1 - ejemplo) * cl_corr
corrections

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
corrections.shape

(300, 300)

In [ ]:
tvclust.sbp().shape

2025-06-02 16:51:55,704 - DEBUG - Calculating the determinant of the covariance


(300, 10)

In [ ]:
tvclust.constraints[tvclust.constraints != 0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
tvclust.ml_correction()

2025-06-02 17:07:44,671 - DEBUG - 
            ML correction: 
                phi_alpha=-0.5772156649015329, 
                phi_beta=-0.5772156649015329, 
                phi_alpha_beta_p=2.3517525890667215, 
                phi_alpha_beta_q=2.3517525890667215
            


0.0

In [ ]:
tvclust.cl_correction()

2025-06-02 17:07:49,937 - DEBUG - 
            CL correction: 
                phi_alpha=2.251752589066721, 
                phi_beta=2.251752589066721, 
                phi_alpha_beta_p=2.3517525890667215, 
                phi_alpha_beta_q=2.3517525890667215
            


0.0

In [ ]:
phi(1) - phi(10 + 1) - phi(10) + phi(1 + 10)

-2.8289682539682537

In [ ]:
corrections = np.ones((300, 300)) * 0.25
responsabilities = np.random.rand(3000).reshape((300, 10))
corrections.dot(responsabilities).shape

(300, 10)

In [ ]:
tvclust.sbp().shape

2025-06-02 17:18:07,953 - DEBUG - Calculating the determinant of the covariance


(300, 10)

In [ ]:
resp = np.ones((300, 10)) / 10
np.sum(resp, axis=0)


array([30., 30., 30., 30., 30., 30., 30., 30., 30., 30.])

In [ ]:
values = X.T.dot(resp)

In [ ]:
beta = np.random.rand(10)

In [ ]:
(values / beta).T

array([[-231.79997062,   80.89334937],
       [-247.05108739,   86.21567066],
       [ -89.14083988,   31.10829171],
       [-100.28942033,   34.99891349],
       [-165.30086079,   57.68654866],
       [-108.69429527,   37.93203933],
       [-102.52366676,   35.77861883],
       [ -96.5206409 ,   33.68368816],
       [ -94.41936657,   32.9503873 ],
       [-314.22667177,  109.65854687]])

In [ ]:
values

array([[-78.47668866, -78.47668866, -78.47668866, -78.47668866,
        -78.47668866, -78.47668866, -78.47668866, -78.47668866,
        -78.47668866, -78.47668866],
       [ 27.38672562,  27.38672562,  27.38672562,  27.38672562,
         27.38672562,  27.38672562,  27.38672562,  27.38672562,
         27.38672562,  27.38672562]])

In [ ]:
beta

array([0.78445643, 0.62764592, 0.96286061, 0.76088448, 0.10114795,
       0.45780052, 0.12037384, 0.32719446, 0.48551938, 0.93798436])

In [ ]:
values

array([[-78.47668866, -78.47668866, -78.47668866, -78.47668866,
        -78.47668866, -78.47668866, -78.47668866, -78.47668866,
        -78.47668866, -78.47668866],
       [ 27.38672562,  27.38672562,  27.38672562,  27.38672562,
         27.38672562,  27.38672562,  27.38672562,  27.38672562,
         27.38672562,  27.38672562]])